In [1]:
import gym

from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy, CnnPolicy, CnnLstmPolicy, CnnLnLstmPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import DQN

from gym.utils import seeding
from stable_baselines.common.env_checker import check_env
import numpy as np

from ads_utils import load_data, plot, Environment
from tqdm import tqdm

from random import randint

/Users/peteskishi/.pyenv/versions/3.6.12/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/peteskishi/.pyenv/versions/3.6.12/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/peteskishi/.pyenv/versions/3.6.12/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/peteskishi/.pyenv/ver

In [2]:
import wandb

In [3]:
sweep_config = {
  "name": "dqn sweep",
  "method": "grid",
  "parameters": {
      "learning_rate": {
            "values": [0.01, 0.005, 0.001, 0.0005, 0.0001, 0.00001]
        },
      "gamma": {
            "values": [0.5, 0.9, 0.99]
        },
      "batch_size": {
          "values": [10, 50, 70, 100]  
        },
      "n_ticks": {
          "values": [1, 5, 20, 50, 150, 200]
      }
    }
}

In [4]:
sweep_id = wandb.sweep(sweep_config, entity="ads", project="dqn-sweep-all-params-val-data")

Create sweep with ID: zi3aya8u
Sweep URL: https://wandb.ai/ads/dqn-sweep-all-params-val-data/sweeps/zi3aya8u


In [5]:
training_data = load_data([i for i in range(1, 12 + 1)])['close'].to_list()
val_data = load_data([i for i in range(13, 18 + 1)])['close'].to_list()

In [6]:
def create_training_env(ticks):
    INITIAL_BALANCE = 10_000
    # sample training data
#     start = randint(0, len(training_data) - 10000 - 1) 
#     sample = training_data[start: start + 10000]
    return Environment(training_data, balance=INITIAL_BALANCE, past_ticks=ticks)

In [7]:
def create_validation_env(ticks):
    INITIAL_BALANCE = 10_000
    val_env = Environment(val_data, balance=INITIAL_BALANCE, past_ticks=ticks)

In [10]:
def train():
    run = wandb.init()
    print("config:", dict(run.config))
    
    N_EPOCH = 100
    env = create_training_env(run.config.n_ticks)
    env.reset()
    model = DQN('MlpPolicy', env, verbose=0, 
                learning_rate=run.config.learning_rate, 
                gamma=run.config.gamma,
                batch_size=run.config.batch_size)
    
    for i in range(N_EPOCH):
        print(f'Epoch {i}')
        model.learn(total_timesteps=10000)

    val_env = create_validation_env(run.config.n_ticks)
    state = val_env.reset(rand_start=False)
    portfolio_values = []
    prices = []
    actions = []

    for i in range(len(val_data)):
        action, _ = model.predict(state)

        price, portfolio_value = env.get_data()
        actions.append(action)
        prices.append(price)
        portfolio_values.append(portfolio_value)

        state, reward, done, _ = env.step(action)
        wandb.log({'portfolio_value': portfolio_value, 'reward': reward})
        if done:
            break

In [ ]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: 6xoxoinv with config:
wandb: 	batch_size: 10
wandb: 	gamma: 0.5
wandb: 	learning_rate: 0.01
wandb: 	n_ticks: 1


config: {'batch_size': 10, 'gamma': 0.5, 'learning_rate': 0.01, 'n_ticks': 1}


Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
